# Watcha 정보 크롤링

In [1]:
# 모듈 불러오기
import pandas as pd
import numpy as np
import time
import re
import requests
import urllib.parse

from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [ ]:
# 왓챠 사이트 여는 함수
def open_watcha():
    
    driver.get(url + keyword)

    time.sleep(3)
    try:
        wait = driver.find_element(By.CSS_SELECTOR, 'div.css-a50hm3.e1yzo30x5 > div > ul > li')
        wait.click()
    except:
        wait = driver.find_element(By.CSS_SELECTOR, 'div.css-1tecamm-ScrollBar.e1yzo30x5 > div > ul > li:nth-child(1)')
        wait.click()
    time.sleep(1)

In [ ]:
# 왓챠 크롤링 함수 만들기
def get_watcha():

    time.sleep(1)

    # 페이지 정보 저장
    raw = driver.page_source
    soup = BeautifulSoup(raw, 'html.parser')

    # 이름
    name = soup.select_one('#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > div > div.css-d3kywm.e11kyjwk3 > div > div.css-1oak90x.e11kyjwk1 > div > div.css-yy4d6f.e1yew28618 > div.css-7jeh73.e1yew28614 > div > h1').text
    if len(name) == 0:
        name = soup.select_one('#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > div > div.css-k0sh94-StyledContent.e11kyjwk3 > div > div.css-1ca4di7-JumbotronBackground.e11kyjwk1 > div > div.css-ysnb50-StyledWallpaperContainer.e1yew28618 > div.css-mkiodl-StyledContentInfoOnWallpaper-createMediaQuery-createMediaQuery-createMediaQuery-createMediaQuery-pageMarginStyle.e1yew28614 > div > h1').text
    
    # 평점
    a = soup.select_one('div.css-1rqwswy.e15eo6m110').text

    # 평점 매긴 사람 수
    tmp_b = soup.select_one('div.css-1jgod02.e15eo6m19').text
    b = re.search(r'(\d{1,3}(,\d{3})*|\d+\.\d+)(만명|명)', tmp_b).group(0)
    
    # 리뷰 수
    c = soup.select_one('div.css-f2umzi.e11kyjwk0 > section:nth-child(2) > header > span').text

    return name, a, b, c

In [ ]:
# 왓챠 정보 크롤링
for idx, row in tqdm_notebook(watcha.loc[851:908].iterrows()):
    tmp_name = []
    
    for i in range(1, int(row['total_season_num']+1)):

        try:

            driver = webdriver.Chrome()
            url = 'https://pedia.watcha.com/ko-KR/search?query='
            name=a='정보없음'; b='정보없음'; c='정보없음'
            
            try:
                keyword = row['title'] + ' 시즌 ' + str(i)
                open_watcha()
                name, a, b, c = get_watcha()
    
            except:
    
                try:
                    keyword = row['english_title'] + ' season ' + str(i)
                    open_watcha()
                    name, a, b, c = get_watcha()
    
                except:
                    if row['total_season_num'] == 1:
    
                        try:
                            keyword = watcha.loc[idx, 'title']
                            open_watcha()
                            name, a, b, c = get_watcha()
                            
                        except:
                            keyword = watcha.loc[idx, 'english_title']
                            open_watcha()
                            name, a, b, c = get_watcha()
                    else:
                        name=a='에러'; b='에러'; c='에러'
            print(name, keyword, a, b, c)
            driver.close()
        except:
            print(name, keyword, a, b, c)
        finally:
            watcha.at[idx, 'season_{}'.format(i)] = [a, b, c]
            tmp_name.append(name)
    watcha.at[idx, 'watcha_name'] = tmp_name